# Advanced options for the DelftaCalculator

This tutorials looks a bit more in-depth at the different options you can pass to the DelftaCalculator. You can refer to the basic tutorial ([delta_vs_direct.ipynb](delta_vs_direct.ipynb)) to get started with the default settings. Again, we'll start with imports: 

In [2]:
import os
import glob
from openbabel.pybel import readfile, readstring
import numpy as np

from delfta.calculator import DelftaCalculator
from delfta.utils import TESTS_PATH, ROOT_PATH

2021/07/15 01:34:10 PM | rdkit | INFO: Enabling RDKit 2020.09.1 jupyter extensions
/Users/Clemens/miniconda3/envs/delfta/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/Clemens/miniconda3/envs/delfta/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/Clemens/miniconda3/envs/delfta/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


The options for the calculator (with their respective defaults) are `tasks="all"`, `delta=True`, `force3d=True`, `addh=True`, `xtbopt=False`, `verbose=True`, and `progress=True`. Let's look at them in detail.

## Tasks

This defines which properties the calculator should predict. You can either pass a list with any combination of the following keys, or simply `"all"` (the default) to get all the values. 

| Property                                      | Key         | Unit  |
|-----------------------------------------------|-------------|-------|
| Formation energy                              | `"E_form"`  | $E_h$ |
| Energy of highest occupied molecular orbital  | `"E_homo"`  | $E_h$ |
| Energy of lowest unoccupied molecular orbital | `"E_lumo"`  | $E_h$ |
| HOMO-LUMO gap                                 | `"E_gap"`   | $E_h$ |
| Molecular dipole                              | `"dipole"`  | D     |
| Mulliken partial charges                      | `"charges"` | $e$   |
| Wiberg bond orders                            | `"wbo"`     | -     |

Note that xTB needs to be run only once for all of them (if `delta=True`, see later), and that HOMO/LUMO/gap energies and the dipole are predicted in a multi-task setting (all via the same network), so the computational cost does not scale linearly with the number of requested properties. 



## Delta

This defines whether or not to use the $\Delta$-prediction approach, *i.e.*, whether to compute the requested values with the semi-empirical GFN2-xTB method, and use the network to predict a correction to this value to obtain an approximation of the DFT value ($\omega$B97X-D/def2-SVP). This is the default (`delta=True`), but you can set it to `False` to directly predict the requested properties from the molecular structure. This removes the need to compute xTB and thus speeds up the process a little bit (though this only makes a noticable differences when you run the calculator for large numbers of molecules).

## force3d

This defines whether you want to use the Merck Molecular Force Field 94 (MMFF94) as implemented in Openbabel to create 3D coordinates for molecules that don't have them. All the quantum mechanical properties that `DelftaCalculator` provides depend on the molecular geometry, so you really shouldn't be passing a 2D molecule and expect reasonable results (of course flat structures like benzene are fine). This defaults to `force3d=True`, and will not affect any molecules you pass that already have a 3D geometry. 

## addh

This defines whether you want to add hydrogens to the molecule. If enabled, we're using Openbabel to check if there's hydrogens missing, and add them accordingly. Just as with `force3d`, it's important to include hydrogens in the molecule rather than using only the heavy atoms in the quantum mechanical calculations/predictions. Note that hydrogens are often omited in SMILES notation. This option also defaults to `addh=True` and won't affect any molecules that already have explicit hydrogens added. 

## xtbopt

This option lets you use GFN2-xTB to optimize the 3D structure of a molecule. This can be useful if you created the conformation with a force field (or used `force3d` to do this), but want to optimize the structure a bit more thoroughly with a more precise method. 

In [26]:
mol = next(readfile("xyz", os.path.join(ROOT_PATH, "notebooks", "check_pdb_files", "wfns_ken", "benzene.xyz")))
print(np.array([atom.coords for atom in mol.atoms]))

[[ 1.6410e-01 -1.3726e+00 -2.0000e-04]
 [-1.1066e+00 -8.2840e-01  1.0000e-04]
 [-1.2707e+00  5.4420e-01 -0.0000e+00]
 [-1.6410e-01  1.3726e+00  0.0000e+00]
 [ 1.1066e+00  8.2840e-01  6.0000e-04]
 [ 1.2707e+00 -5.4420e-01 -7.0000e-04]
 [ 2.9230e-01 -2.4449e+00  4.4000e-03]
 [-1.9712e+00 -1.4756e+00  0.0000e+00]
 [-2.2635e+00  9.6930e-01 -5.0000e-04]
 [-2.9230e-01  2.4449e+00 -8.0000e-04]
 [ 1.9712e+00  1.4756e+00 -1.0000e-04]
 [ 2.2635e+00 -9.6930e-01  2.0000e-04]]


In [29]:
calc_delta = DelftaCalculator(tasks="all", delta=True, xtbopt=True) 
predictions_delta = calc_delta.predict(mol)

2021/07/15 01:40:29 PM | DelFTa | INFO: Now running xTB...
100%|██████████| 1/1 [00:00<00:00,  8.98it/s]
2021/07/15 01:40:29 PM | DelFTa | INFO: Now running network for model multitask_delta...
100%|██████████| 1/1 [00:00<00:00, 50.63it/s]
2021/07/15 01:40:29 PM | DelFTa | INFO: Now running network for model single_energy_delta...
100%|██████████| 1/1 [00:00<00:00, 56.15it/s]
2021/07/15 01:40:29 PM | DelFTa | INFO: Now running network for model charges_delta...
100%|██████████| 1/1 [00:00<00:00, 67.94it/s]


In [31]:
print(np.array([atom.coords for atom in mol.atoms]))

[[ 1.6410e-01 -1.3726e+00 -2.0000e-04]
 [-1.1066e+00 -8.2840e-01  1.0000e-04]
 [-1.2707e+00  5.4420e-01 -0.0000e+00]
 [-1.6410e-01  1.3726e+00  0.0000e+00]
 [ 1.1066e+00  8.2840e-01  6.0000e-04]
 [ 1.2707e+00 -5.4420e-01 -7.0000e-04]
 [ 2.9230e-01 -2.4449e+00  4.4000e-03]
 [-1.9712e+00 -1.4756e+00  0.0000e+00]
 [-2.2635e+00  9.6930e-01 -5.0000e-04]
 [-2.9230e-01  2.4449e+00 -8.0000e-04]
 [ 1.9712e+00  1.4756e+00 -1.0000e-04]
 [ 2.2635e+00 -9.6930e-01  2.0000e-04]]
